### Metodos númericos. Sistema de Ecuaciones no lineales

Recordar ejemplos clase anterior.

- Ejemplo: Circunferencia de radio 2 y recta y=x. El sistema de ecuaciones no lineal seria:

$$ x^2 + y^2 = 4 $$
$$ x - y = 0 $$

Solucion encontrada: $\sqrt{2}$

Usaremos https://docs.scipy.org/doc/scipy/reference/optimize.html

In [4]:
#alternativa 1
from scipy.optimize import newton

def f(x):
    return x**2 - 2

def df(x):
    return 2 * x

sol = newton(f, x0=1, fprime=df, tol=1e-8, full_output=True)
print(sol)

(np.float64(1.4142135623730951),       converged: True
           flag: converged
 function_calls: 10
     iterations: 5
           root: 1.4142135623730951
         method: newton)


Hay multiples metodos, ver **root finding, scalar functions** en https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html#scipy.optimize.root_scalar

In [6]:
# alternativa 2... se agrupan todos los metodos con root_scalar
from scipy.optimize import root_scalar

def f(x):
    return x**2 - 2

def df(x):
    return 2 * x

sol1 = root_scalar(f, bracket=[0, 2], method='bisect')
print("Biseccion:", sol1)
print("---------------------")
sol2 = root_scalar(f, x0=1, fprime=df, method='newton')
print("Newton:", sol2)
print("---------------------")
sol2 = root_scalar(f, x0=0, x1=1, method='secant')
print("Secante:", sol2)
# Hay mas metodos... investigar!!!!

Biseccion:       converged: True
           flag: converged
 function_calls: 42
     iterations: 40
           root: 1.4142135623715149
         method: bisect
---------------------
Newton:       converged: True
           flag: converged
 function_calls: 10
     iterations: 5
           root: 1.4142135623730951
         method: newton
---------------------
Secante:       converged: True
           flag: converged
 function_calls: 9
     iterations: 8
           root: 1.414213562373095
         method: secant


In [7]:
#alternativa 3, estilo sistema de ecuaciones

import numpy as np
from scipy.optimize import fsolve

# Sistema de ecuaciones
def f(r):
    x, y = r
    equation1 = x**2 + y**2 - 4
    equation2 = x - y
    return np.array([equation1, equation2])

# Matriz jacobiana (la derivada)
def jacobian(r):
    x, y = r
    d_equation1_dx = 2*x
    d_equation1_dy = 2*y
    d_equation2_dx = 1
    d_equation2_dy = -1
    return np.array([[d_equation1_dx, d_equation1_dy], [d_equation2_dx, d_equation2_dy]])

# Punto inicial para la solucion, cambiarlo a 0,0 y ver que pasa
r0 = np.array([1.0, 0.0])

# Using fsolve with Jacobian
solution = fsolve(func=f, x0=r0, fprime=jacobian, xtol=1e-08)

print(solution, f(solution))

[1.41421356 1.41421356] [2.66453526e-15 0.00000000e+00]


Existen más metodos como Broyden, Newton-Krylov, Anderson, etc... ver en: **root finding, general nonlinear solvers**. de scipy.optimize. Del mismo modo, como existe root_scalar con todos los metodos, existe root, con el metodo para resolver al estilo fsolve.

In [12]:
from scipy.optimize import root
sol = root(f, r0, method='krylov')
print(sol)
print(sol.x[0])

 message: A solution was found at the specified tolerance.
 success: True
  status: 1
     fun: [-5.009e-06  6.171e-07]
       x: [ 1.414e+00  1.414e+00]
     nit: 36
  method: krylov
    nfev: 198
1.414212985511528


Al igual que la semana pasada.... la sympy nos puede ayudar, sin embargo es solo referencia.

In [16]:
from sympy import var, solve
import time

inicio = time.time()

x, y = var('x y')

f1 = x**2+y**2-4
f2 = x-y

sols = solve((f1, f2), (x, y))
fin = time.time()
print("Tiempo sympy:", fin-inicio)

print(sols)

Tiempo sympy: 0.011548280715942383
[(-sqrt(2), -sqrt(2)), (sqrt(2), sqrt(2))]


### Desafio (Tarea 4). La función de Cobb-Douglas:

En economía es muy famosa la función de producción de *Cobb-Douglas*, la cual veremos (eso espero) en el curso de **IIP314W Optimización aplicada a negocios**, puede revisarla en:

https://es.wikipedia.org/wiki/Funci%C3%B3n_de_producci%C3%B3n_de_Cobb-Douglas

Usted debe solucionar el siguiente problema, utilizando las rutinas de la librerias scipy.optimize que apliquen al problema, probando todos los metodos distintos que pueda, calculando tiempos de ejecución y diferencias relativas entre las soluciones. En este trabajo, no es necesario utilizar las rutinas propias que vimos en las dos clases anteriores. Entonces,

---

### Desafio 4

Consideremos una economía con **dos sectores productivos** (i = 1, 2).  
Cada sector i tiene una función de producción Cobb–Douglas:

$$Y_i = A_i \, K_i^{\alpha_i} \, L_i^{1-\alpha_i}$$

donde:
- $K_i$: capital asignado al sector i  
- $L_i$: trabajo asignado al sector i  
- $A_i$: productividad del sector i
- $\alpha_i$: elasticidad del capital en el sector i.

El capital y el trabajo totales son conocidos:

$$K_1 + K_2 = \bar K, \qquad L_1 + L_2 = \bar L.$$

### Condiciones de Equilibrio

En mercados competitivos, el costo de capital $r$ y el salario de trabajo $w$ se igualan a los productos marginales en cada sector:

$$r = \frac{\partial Y_i}{\partial K_i} \qquad i = 1,2.$$

$$w = \frac{\partial Y_i}{\partial L_i} \qquad i = 1,2.$$

Esto se debe a que en un mercado competitivo, todos los agentes (sectores, empresas) pueden mover capital y trabajo libremente. Si un sector pagara un costo mayor por el capital que otro, el capital migraría hacia ese sector hasta que los precios se igualen. Lo mismo con los salarios: si un sector pagara más por el trabajo, los trabajadores se moverían allí, hasta que los salarios se equilibren (supongamos que en Chile pasa eso).

Por eso, en equilibrio, existe un único *r* y un único *w* para toda la economía.

Esto da lugar a un sistema de 4 ecuaciones no lineales en las 4 incógnitas:


$$\begin{cases}
f_1(K_1,L_1,r,w) = r - A_1 \alpha_1 K_1^{\alpha_1-1} L_1^{1-\alpha_1} = 0, \\[6pt]
f_2(K_1,L_1,r,w) = r - A_2 \alpha_2 (K_2)^{\alpha_2-1} (L_2)^{1-\alpha_2} = 0, \\[6pt]
f_3(K_1,L_1,r,w) = w - A_1 (1-\alpha_1) K_1^{\alpha_1} L_1^{-\alpha_1} = 0, \\[6pt]
f_4(K_1,L_1,r,w) = w - A_2 (1-\alpha_2) (K_2)^{\alpha_2} (L_2)^{-\alpha_2} = 0.
\end{cases}
$$

Nota: Debemos reemplazar $K_2 = \bar K - K_1$ y $L_2 = \bar L - L_1\$.

Solucione el sistema de ecuaciones con los siguientes datos:
$$
A_1 = 1.2, \quad A_2 = 0.9, \quad \alpha_1 = 0.3, \quad \alpha_2 = 0.6, 
\quad \bar K = 100, \quad \bar L = 50.
$$

Como solución inicial, empiece con alguna que le parezca razonable, si quiere puede usar la mia (abajo).

---

Realice un informe donde:

- Compare los resultados y tiempos de ejecución. Comente las diferencias de la variable *sol* de abajo.
- Modifique un poco la solución inicial o parametros, vea si sigue funcionando o empieza a funcionar.
- Investigue qué interpretación económica tienen las variables *r* y *w* encontradas.

In [2]:
# Ayuda... hay convergencia a veces... y espero que lo hice bien...
import numpy as np
from scipy.optimize import root

# parámetros
A1, A2 = 1.2, 0.9
alpha1, alpha2 = 0.3, 0.6
Kbar, Lbar = 100.0, 50.0

def F(x):
    K1, L1, r, w = x
    K2 = Kbar - K1
    L2 = Lbar - L1
    
    # ecuaciones de equilibrio
    f1 = r - A1*alpha1*(np.sign(K1) * (np.abs(K1))**(alpha1-1))*(np.sign(L1) * (np.abs(L1))**(1-alpha1))
    f2 = r - A2*alpha2*(K2**(alpha2-1))*(L2**(1-alpha2))
    f3 = w - 1*A1*(1-alpha1)*(np.sign(K1))*((np.abs(K1))**alpha1)*(L1**(-alpha1))
    f4 = w - A2*(1-alpha2)*(K2**alpha2)*(L2**(-alpha2))
    
    return [f1, f2, f3, f4]

# condición inicial
x0 = [50, 25, 0.5, 0.5]

# solucion del sistema no lineal
sol = root(F, x0, method='lm')
#sol = root(F, x0)

print(sol)
print("---------------")
print("K1 = %.6f, L1 = %.6f, r = %.6f, w = %.6f" % tuple(sol.x))

# capital y trabajo del segundo sector
K2, L2 = Kbar - sol.x[0], Lbar - sol.x[1]
print("K2 = %.6f, L2 = %.6f" % (K2, L2))

 message: The relative error between two consecutive iterates is at most 0.000000
 success: True
  status: 2
     fun: [-5.551e-17  0.000e+00  0.000e+00 -1.110e-16]
       x: [ 5.628e+01  4.092e+01  2.880e-01  9.243e-01]
   cov_x: [[ 7.890e+05  2.095e+05 -1.186e+03  2.625e+03]
           [ 2.095e+05  5.601e+04 -3.165e+02  7.080e+02]
           [-1.186e+03 -3.165e+02  2.289e+00 -3.990e+00]
           [ 2.625e+03  7.080e+02 -3.990e+00  9.536e+00]]
  method: lm
    nfev: 43
    fjac: [[-1.414e+00  0.000e+00  7.071e-01  7.071e-01]
           [ 0.000e+00 -1.414e+00 -5.000e-01 -5.000e-01]
           [ 3.839e-02 -5.487e-03 -4.957e-02 -5.068e-01]
           [-5.487e-03 -6.694e-04  1.316e-02  1.126e-03]]
    ipvt: [3 2 1 0]
     qtf: [ 3.925e-15  6.280e-16 -1.314e-14 -2.509e-15]
---------------
K1 = 56.283360, L1 = 40.919170, r = 0.287995, w = 0.924305
K2 = 43.716640, L2 = 9.080830


C:\Users\bvial\AppData\Local\Temp\ipykernel_14496\1165745367.py:18: RuntimeWarning: invalid value encountered in scalar power
  f3 = w - 1*A1*(1-alpha1)*(np.sign(K1))*((np.abs(K1))**alpha1)*(L1**(-alpha1))
